# Visual SLAM

1. Initialization: At first frame, initialize map with 3D points from stereo.
2. Tracking:
   - at frame i+1, match keypoints between i and i+1


In [ ]:
import os
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

from lightglue import LightGlue, SuperPoint
from lightglue.utils import rbd

from lac.slam.feature_tracker import FeatureTracker
from lac.perception.depth import project_pixel_to_rover
from lac.utils.frames import apply_transform
from lac.utils.plotting import plot_3d_points, plot_surface, plot_poses, plot_path_3d
from lac.util import load_data, load_stereo_images
from lac.params import LAC_BASE_PATH, DT

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/slam_map1_preset1_teleop"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Num poses: {len(poses)}")

# Load the images
left_imgs, right_imgs = load_stereo_images(data_path)

map = np.load(
    Path(LAC_BASE_PATH) / "data/heightmaps/competition/Moon_Map_01_preset_0.dat",
    allow_pickle=True,
)

In [ ]:
img_idxs = sorted(list(left_imgs.keys()))

In [ ]:
fig = plot_surface(map)
fig = plot_poses(poses[::20], fig=fig)
fig.show()

## Stereo (PnP) VO


In [ ]:
from lac.slam.visual_odometry import StereoVisualOdometry

In [ ]:
svo = StereoVisualOdometry(cam_config)

In [ ]:
start_idx = 80
svo.initialize(poses[start_idx], left_imgs[start_idx], right_imgs[start_idx])

In [ ]:
svo_poses = [poses[start_idx]]
pose_deltas = []

end_idx = img_idxs[-1]
# end_idx = 2000

for idx in tqdm(np.arange(start_idx + 2, end_idx, 2)):
    svo.track(left_imgs[idx], right_imgs[idx])
    svo_poses.append(svo.rover_pose)
    pose_deltas.append(svo.pose_delta)

In [ ]:
fig = plot_poses(poses, no_axes=True, color="black", name="Ground Truth")
fig = plot_poses(svo_poses, no_axes=True, fig=fig, color="red", name="Stereo (PnP) VO")
fig.show()

In [ ]:
fig.write_html("vo_traj.html")

Ground constraints


In [ ]:
# Search over the poses for poses where XY are within 0.01m of each other
positions_2d = np.array([pose[:2, 3] for pose in svo_poses])


def compute_distance_matrix(points):
    # Compute the squared Euclidean distance matrix
    diff = points[:, np.newaxis, :] - points[np.newaxis, :, :]
    dist_matrix = np.sqrt(np.sum(diff**2, axis=2))
    return dist_matrix


dist_matrix = compute_distance_matrix(positions_2d)

In [ ]:
# Find the indices of the pairs of points that are within 0.01m
threshold = 0.01
indices = np.argwhere(dist_matrix < threshold)
# Filter out pairs where the indices are the same
indices = indices[indices[:, 0] != indices[:, 1]]
# Remove duplicates (i.e., (i, j) and (j, i))
unique_indices = set()
for i, j in indices:
    if abs(i - j) > 100 and (j, i) not in unique_indices:
        unique_indices.add((i, j))
# Convert the set back to a list of tuples
unique_indices = list(unique_indices)

In [ ]:
unique_indices

In [ ]:
import plotly.graph_objects as go

fig = plot_poses(poses, no_axes=True, color="black", name="Ground Truth")
fig = plot_poses(svo_poses, no_axes=True, fig=fig, color="red", name="Stereo (PnP) VO")
# Add the lines between the points
for i, j in unique_indices:
    fig.add_trace(
        go.Scatter3d(
            x=[svo_poses[i][0, 3], svo_poses[j][0, 3]],
            y=[svo_poses[i][1, 3], svo_poses[j][1, 3]],
            z=[svo_poses[i][2, 3], svo_poses[j][2, 3]],
            mode="markers+lines",
            marker=dict(size=5, color="blue"),
            line=dict(color="blue", width=5),
            showlegend=False,
        )
    )
fig.show()

In [ ]:
import gtsam
from gtsam.symbol_shorthand import B, V, X

graph = gtsam.NonlinearFactorGraph()
values = gtsam.Values()

svo_pose_sigma = 1e-2 * np.ones(6)
svo_pose_noise = gtsam.noiseModel.Diagonal.Sigmas(svo_pose_sigma)

values.insert(X(0), gtsam.Pose3(svo_poses[0]))
graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(svo_poses[0])))

for i in range(1, len(svo_poses)):
    values.insert(X(i), gtsam.Pose3(svo_poses[i]))
    graph.push_back(
        gtsam.BetweenFactorPose3(X(i - 1), X(i), gtsam.Pose3(pose_deltas[i - 1]), svo_pose_noise)
    )

translation_only_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([1e6, 1e6, 1e6, 0.1, 0.1, 0.1]))
for i, j in unique_indices:
    # Add a prior on the relative pose between the two poses
    graph.push_back(gtsam.BetweenFactorPose3(X(i), X(j), gtsam.Pose3(np.eye(4)), svo_pose_noise))

In [ ]:
params = gtsam.LevenbergMarquardtParams()
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, values, params)
result = optimizer.optimize()

opt_poses = [result.atPose3(X(i)).matrix() for i in range(len(svo_poses))]

In [ ]:
fig = plot_poses(poses, no_axes=True, color="black", name="Ground Truth")
fig = plot_poses(svo_poses, no_axes=True, fig=fig, color="red", name="Stereo (PnP) VO")
fig = plot_poses(opt_poses, no_axes=True, fig=fig, color="green", name="Stereo (PnP) VO")
for i, j in unique_indices:
    fig.add_trace(
        go.Scatter3d(
            x=[opt_poses[i][0, 3], opt_poses[j][0, 3]],
            y=[opt_poses[i][1, 3], opt_poses[j][1, 3]],
            z=[opt_poses[i][2, 3], opt_poses[j][2, 3]],
            mode="markers+lines",
            marker=dict(size=5, color="blue"),
            line=dict(color="blue", width=5),
            showlegend=False,
        )
    )
fig.show()

In reality, the XYs will change during optimization, so we need a global loss over the whole trajectory
that enforces Z to be close when XY is close. We could do this through an energy term such as:
$$ E*{\text{smooth}} = \sum*{i,j} w(\| (x_i, y_i) - (x_j, y_j) \|)(z_i - z_j)^2 $$
where $w(\cdot)$ is a kernel (e.g. Gaussian) that assigns high weight when xy distance is small and quickly
decays when distance exceeds a threshold.


Loosely coupled fusion with IMU odometry


In [ ]:
from lac.localization.imu_recovery import ImuEstimator
from lac.utils.frames import invert_transform_mat

imu_estimator = ImuEstimator(initial_pose)
imu_recovery_poses = [initial_pose]
imu_recovery_deltas = []
gt_pose_deltas = []

for i in tqdm(range(len(imu_data))):
    imu_estimator.update(imu_data[i], exact=False)
    imu_recovery_poses.append(imu_estimator.get_pose())
    imu_recovery_deltas.append(imu_estimator.get_pose_delta())
    gt_pose_deltas.append(poses[i + 1] @ invert_transform_mat(poses[i]))

In [ ]:
len(svo_poses), len(pose_deltas), len(imu_recovery_deltas)

In [ ]:
import gtsam
from gtsam.symbol_shorthand import B, V, X

graph = gtsam.NonlinearFactorGraph()
values = gtsam.Values()

svo_pose_sigma = 1e-2 * np.ones(6)
svo_pose_noise = gtsam.noiseModel.Diagonal.Sigmas(svo_pose_sigma)
imu_pose_sigma = 1e-2 * np.ones(6)
imu_pose_noise = gtsam.noiseModel.Diagonal.Sigmas(imu_pose_sigma)

values.insert(X(0), gtsam.Pose3(svo_poses[0]))
graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(svo_poses[0])))

for i in range(1, len(svo_poses)):
    values.insert(X(i), gtsam.Pose3(svo_poses[i]))
    graph.push_back(
        gtsam.BetweenFactorPose3(X(i - 1), X(i), gtsam.Pose3(pose_deltas[i - 1]), svo_pose_noise)
    )
    step = 2 * i + start_idx - 1
    imu_delta = imu_recovery_deltas[step] @ imu_recovery_deltas[step - 1]
    graph.push_back(
        gtsam.BetweenFactorPose3(X(i - 1), X(i), gtsam.Pose3(imu_delta), imu_pose_noise)
    )

In [ ]:
params = gtsam.LevenbergMarquardtParams()
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, values, params)
result = optimizer.optimize()

In [ ]:
opt_poses = []
for i in range(len(svo_poses)):
    opt_poses.append(result.atPose3(X(i)).matrix())

In [ ]:
fig = plot_poses(poses, no_axes=True, color="black", name="Ground Truth")
fig = plot_poses(svo_poses, no_axes=True, fig=fig, color="red", name="Stereo (PnP) VO")
fig = plot_poses(opt_poses, no_axes=True, fig=fig, color="green", name="Stereo (PnP) VO")
fig.show()

In [ ]:
fig.write_html("stereo_pnp_vo_full_spiral.html")

# SymForce


In [ ]:
import time
import symforce

try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass

from lac.localization.factor_graph import FactorGraph

In [ ]:
from lac.localization.imu_recovery import ImuEstimator
from lac.utils.frames import invert_transform_mat

imu_estimator = ImuEstimator(initial_pose)
imu_recovery_poses = [initial_pose]
imu_recovery_deltas = []
gt_pose_deltas = []

for i in tqdm(range(len(imu_data))):
    imu_estimator.update(imu_data[i], exact=False)
    imu_recovery_poses.append(imu_estimator.get_pose())
    imu_recovery_deltas.append(imu_estimator.get_pose_delta())
    gt_pose_deltas.append(poses[i + 1] @ invert_transform_mat(poses[i]))

In [ ]:
len(imu_recovery_deltas), len(poses)

In [ ]:
tracker = FeatureTracker(cam_config)
graph = FactorGraph()

START_FRAME = 80
tracker.initialize(initial_pose, left_imgs[START_FRAME], right_imgs[START_FRAME])

In [ ]:
N = 1000
UPDATE_RATE = 10
WINDOW_SIZE = 20

curr_pose = initial_pose
graph.add_pose(0, initial_pose)

# i is step which is 0 for initial and starts at 1 for the first run_step call
for i in tqdm(range(1, N)):
    if i % UPDATE_RATE == 0:
        noisy_pose = poses[i].copy()
        noisy_pose[:3, 3] = noisy_pose[:3, 3] + np.random.normal(0, 0.1, 3)
        graph.add_pose(i, poses[i])
        # graph.add_pose(i, imu_recovery_poses[i])

    # Add IMU factors
    # if i > 1:
    #     graph.add_accel_factor(i, imu_data[i - 1][:3])
    # graph.add_gyro_factor(i, imu_data[i - 1][3:])

    # Add vision factors
    if i % 2 == 0 and i > START_FRAME:
        if i % 10 == 0:
            tracker.track_keyframe(poses[i], left_imgs[i], right_imgs[i])
        else:
            tracker.track(left_imgs[i])

        if i % UPDATE_RATE == 0:
            for k in range(len(tracker.track_ids)):
                graph.add_reprojection_factor(
                    i, tracker.prev_pts[k], tracker.world_points[k], tracker.track_ids[k]
                )

    # Sliding window optimization
    # if i % UPDATE_RATE == 0:
    #     result = graph.optimize(window=(i - UPDATE_RATE, i))
    #     curr_pose = graph.get_pose(i)

In [ ]:
# Batch optimization
result = graph.optimize(verbose=True)
fgo_poses = graph.get_all_poses()

In [ ]:
fig = plot_poses(poses[:N], no_axes=True, color="black", name="Ground truth")
fig = plot_poses(fgo_poses[:N], fig=fig, no_axes=True, color="green", name="FGO")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

# GTSAM vision factors


In [ ]:
import gtsam
from gtsam.symbol_shorthand import X

from gtsam import Pose3

from lac.localization.gtsam_factor_graph import GtsamFactorGraph

In [ ]:
tracker = FeatureTracker(cam_config)

START_FRAME = 80
initial_pose = poses[START_FRAME]
tracker.initialize(poses[START_FRAME], left_imgs[START_FRAME], right_imgs[START_FRAME])
# tracker.initialize(initial_pose, left_imgs[START_FRAME], right_imgs[START_FRAME])

In [ ]:
graph = GtsamFactorGraph()

In [ ]:
N = 4000
UPDATE_RATE = 10

idx = 0
curr_pose = initial_pose
graph.add_pose(idx, initial_pose)
graph.add_vision_factors(idx, tracker.world_points, tracker.prev_pts, tracker.track_ids)


# i is step which is 0 for initial and starts at 1 for the first run_step call
for i in tqdm(range(2, N)):
    step = i + START_FRAME

    # Run tracker
    if i % 2 == 0:
        if i % 10 == 0:
            tracker.track_keyframe(poses[step], left_imgs[step], right_imgs[step])
        else:
            tracker.track(left_imgs[step])

    # Add new pose and vision factors to graph
    if i % UPDATE_RATE == 0:
        idx += 1
        noisy_pose = poses[step].copy()
        noisy_pose[:3, 3] += np.random.normal(0, 0.0, 3)
        graph.add_pose(idx, noisy_pose)
        # graph.add_pose(idx, poses[step])
        # graph.add_pose_prior(idx, noisy_pose)
        graph.add_vision_factors(idx, tracker.world_points, tracker.prev_pts, tracker.track_ids)

In [ ]:
result = graph.optimize()

In [ ]:
print("initial error = {}".format(graph.graph.error(graph.initial_estimate)))
print("final error = {}".format(graph.graph.error(result)))

In [ ]:
initial_poses = []
result_poses = []

for i in range(idx):
    initial_poses.append(graph.initial_estimate.atPose3(X(i)).matrix())
    result_poses.append(result.atPose3(X(i)).matrix())

In [ ]:
fig = plot_poses(poses[: N + START_FRAME], no_axes=True, color="black", name="Ground truth")
fig = plot_poses(initial_poses, fig=fig, no_axes=True, color="orange", name="GTSAM initial poses")
fig = plot_poses(result_poses, fig=fig, no_axes=True, color="green", name="GTSAM optimized poses")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
np.round(result.atPose3(X(0)).matrix(), 5)

In [ ]:
np.round(poses[START_FRAME], 5)

# GTSAM VIO


In [ ]:
import gtsam
from gtsam.symbol_shorthand import X, L

from gtsam import Pose3, PriorFactorPose3

from lac.slam.gtsam_factor_graph import GtsamVIO

In [ ]:
# tracker = FeatureTracker(cam_config)
tracker = FeatureTracker(cam_config, max_keypoints=2048, max_stereo_matches=1000)

START_FRAME = 80
initial_pose = poses[START_FRAME]
tracker.initialize(poses[START_FRAME], left_imgs[START_FRAME], right_imgs[START_FRAME])

In [ ]:
vio = GtsamVIO(fix_landmarks=False)

# Initialize with 2 (stationary) poses
idx = 0
vio.add_pose(idx, initial_pose)
vio.add_vision_factors(idx, tracker.world_points, tracker.prev_pts, tracker.track_ids)
idx += 1
vio.add_pose(idx, initial_pose)
vio.add_vision_factors(idx, tracker.world_points, tracker.prev_pts, tracker.track_ids)
latest_pose = initial_pose

In [ ]:
N = 4000
IMG_RATE = 2
KEYFRAME_RATE = 10

for i in tqdm(range(4, N, IMG_RATE)):
    step = i + START_FRAME
    next_pose = latest_pose @ imu_recovery_deltas[step - 2] @ imu_recovery_deltas[step - 1]

    # Run tracker
    tracker.track_keyframe(next_pose, left_imgs[step], right_imgs[step])
    # if i % KEYFRAME_RATE == 0:
    #     tracker.track_keyframe(next_pose, left_imgs[step], right_imgs[step])
    #     # TODO: we should probably add (or update) new keyframe after optimizing
    # else:
    #     tracker.track(left_imgs[step])

    # Add new pose and vision factors to graph each frame
    idx += 1
    vio.add_pose(idx, next_pose)
    vio.add_vision_factors(idx, tracker.world_points, tracker.prev_pts, tracker.track_ids)
    result = vio.optimize(verbose=False)
    latest_pose = result.atPose3(X(idx)).matrix()

In [ ]:
vio_poses = list(vio.poses.values())

In [ ]:
fig = plot_poses(poses[: N + START_FRAME], no_axes=True, color="black", name="Ground truth")
fig = plot_poses(vio_poses, fig=fig, no_axes=True, color="green", name="GTSAM VIO")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
fig.write_html("gtsam_vio.html")

In [ ]:
from lac.util import positions_rmse_from_poses, rotations_rmse_from_poses


## GTSAM IMU factors


In [ ]:
from gtsam import imuBias, noiseModel, PriorFactorConstantBias
from gtsam.symbol_shorthand import B, V, X, L

In [ ]:
g = 1.622
IMU_PARAMS = gtsam.PreintegrationParams.MakeSharedU(g)
# I = np.eye(3)
# IMU_PARAMS.setAccelerometerCovariance(I * 0.2)
# IMU_PARAMS.setGyroscopeCovariance(I * 0.2)
# IMU_PARAMS.setIntegrationCovariance(I * 0.2)
gyro_sigma = 1e-3
accel_sigma = 1e-3
I_3x3 = np.eye(3)
IMU_PARAMS.setGyroscopeCovariance(gyro_sigma**2 * I_3x3)
IMU_PARAMS.setAccelerometerCovariance(accel_sigma**2 * I_3x3)
IMU_PARAMS.setIntegrationCovariance(1e-7**2 * I_3x3)

In [ ]:
graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

pose_noise = gtsam.noiseModel.Diagonal.Sigmas(0.2 * np.ones(6))
# graph.push_back(gtsam.PriorFactorPose3((X(0), gtsam.Pose3(poses[0])), pose_noise))
# Fix first pose
graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(poses[0])))
initial_estimate.insert(X(0), gtsam.Pose3(poses[0]))

# IMU prior
BIAS_KEY = B(0)
zero_bias = imuBias.ConstantBias(np.zeros(3), np.zeros(3))
graph.add(PriorFactorConstantBias(BIAS_KEY, zero_bias, noiseModel.Constrained.All(6)))
initial_estimate.insert(BIAS_KEY, zero_bias)

velocity_key = V(0)
velocity_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.2)
velocity_0 = np.array([0.0, 0.0, 0])
graph.push_back(gtsam.PriorFactorVector(velocity_key, velocity_0, velocity_noise))
initial_estimate.insert(velocity_key, velocity_0)

# Preintegrator
accum = gtsam.PreintegratedImuMeasurements(IMU_PARAMS)

n_frames = 1000

for i in range(1, n_frames):
    accum.integrateMeasurement(imu_data[i, :3], imu_data[i, 3:], DT)

    # Initialize with noisy ground truth poses
    # initial_estimate.insert(pose_key, gtsam.Pose3(poses[i]).compose(DELTA))
    initial_estimate.insert(X(i), gtsam.Pose3(poses[i]))
    initial_estimate.insert(V(i), np.array([0.0, 0.0, 0]))

    graph.add(gtsam.ImuFactor(X(i - 1), V(i - 1), X(i), V(i), BIAS_KEY, accum))

    accum.resetIntegration()

In [ ]:
params = gtsam.LevenbergMarquardtParams()
# params.setMaxIterations(100)
# params.setlambdaUpperBound(1.e+6)
# params.setlambdaLowerBound(0.1)
# params.setDiagonalDamping(1000)
# params.setVerbosity('ERROR')
# params.setVerbosityLM('SUMMARY')
# params.setRelativeErrorTol(1.e-9)
# params.setAbsoluteErrorTol(1.e-9)

In [ ]:
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()
# for i in range(10):
#     print(f"Iteration {i + 1}, Total Error: {graph.error(optimizer.values())}")
#     optimizer.iterate()

In [ ]:
est_traj = np.array([result.atPose3(X(k)).translation() for k in range(n_frames)])

fig = plot_poses(poses[:n_frames], no_axes=True, color="black", name="Ground Truth")
fig = plot_path_3d(est_traj, fig=fig, color="red", name="GTSAM IMU")
fig.show()